## Peer-graded Assignment week 3, notebook 3 of 3: Segmenting and Clustering Neighbourhoods in Toronto
This notebook is used for assignment week 3, Datascience Capstone.
At first, I copy relevant parts from notebook 1 and 2:

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np

In [2]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
table=soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighbourhood"])
df1=df[df.Borough != 'Not assigned']
df2=df1.groupby(["Postcode","Borough"])['Neighbourhood'].apply(','.join).reset_index()
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned', df2['Borough'], df2['Neighbourhood'])
geodata = pd.read_csv("https://cocl.us/Geospatial_data") 
geodata.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df3 = pd.merge(df2, geodata, on='Postcode')
df3.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Just to double-check that everything is OK with the dataframe (I expect 103 rows and 5 columns):

In [3]:
df3.shape

(103, 5)

Downloading/importing all the packages I need for this exercise:

In [4]:
# convert an address into latitude and longitude values
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
!pip install matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
!pip install folium
import folium

# library to handle JSON files
import json

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

### Explore the whole dataset for Toronto

I begin with finding Toronto's coordinates:

In [5]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Creating a map of Toronto with unique postcodes superimposed on top:

In [6]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, postcode, borough, neighbourhood in zip(df3['Latitude'], df3['Longitude'], df3['Postcode'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}, {}'.format(postcode, borough, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Explore boroughs that contain the word "Toronto"

Before I slice the data, I would like to know how many and which boroughs there are in the dataset and how many different postcodes there are in each of them:

In [7]:
df3.groupby('Borough')['Postcode'].nunique()

Borough
Central Toronto      9
Downtown Toronto    18
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Queen's Park         1
Scarborough         17
West Toronto         6
York                 5
Name: Postcode, dtype: int64

 Now I create a new dataframe (b_toronto) with only boroughs that contain the word Toronto:

In [8]:
b_toronto= df3 [df3['Borough'].str.contains("Toronto") == True].reset_index(drop=True)
b_toronto.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


I expect the dataset to have 9+18+5+6=38 rows and 5 columns:

In [9]:
b_toronto.shape

(38, 5)

Now I create a new map in order to visualize the new dataframe:

In [10]:
# create map of boroughs that include the word Toronto using latitude and longitude values
map_b_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(b_toronto['Latitude'], b_toronto['Longitude'], b_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_b_toronto)  
    
map_b_toronto

To define Foursquare credentials and version:

In [11]:
CLIENT_ID = 'RWLY0RHFLLB2X5L5SF2FV243QOFFSINLN0HZUOT1BFEBCK5G' # your Foursquare ID
CLIENT_SECRET = 'SBFSJDUVKAH2EJRJFRDS1ZYIQYLTKOTXAYREQN5AENT4L5PA' # your Foursquare Secret
VERSION = '20190623' # Foursquare API version

To create a function that gets venues for all the boroughs in b_toronto: 

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

To run the function above on each neighbourhood in the b_toronto dataset:

In [13]:
b_toronto_venues = getNearbyVenues(names=b_toronto['Neighbourhood'],
                                   latitudes=b_toronto['Latitude'],
                                   longitudes=b_toronto['Longitude']
                                  )


The Beaches


NameError: name 'LIMIT' is not defined

To check the size and first 20 rows of the resulting dataframe:

In [ ]:
b_toronto_venues.shape

In [ ]:
b_toronto_venues.head(20)

To check how many venues were returned for each neighbourhood:

In [ ]:
b_toronto_venues.groupby('Neighbourhood').count()

To find out how many unique categories can be curated from all the returned venues:

In [ ]:
print('There are {} uniques categories.'.format(len(b_toronto_venues['Venue Category'].unique())))

To analyze each neighbourhood:

In [ ]:
# one hot encoding
b_toronto_onehot = pd.get_dummies(b_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
b_toronto_onehot['Neighbourhood'] = b_toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [b_toronto_onehot.columns[-1]] + list(b_toronto_onehot.columns[:-1])
b_toronto_onehot = b_toronto_onehot[fixed_columns]

b_toronto_onehot.head()

To check the dataframe's size:

In [ ]:
b_toronto_onehot.shape

To group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category:

In [ ]:
b_toronto_grouped = b_toronto_onehot.groupby('Neighbourhood').mean().reset_index()
b_toronto_grouped.head()

The new size:

In [ ]:
b_toronto_grouped.shape

In [ ]:
b_toronto_grouped.head()

To return top 8 venues (just to try something else than 10) for each group of neighbourhoods:

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 8

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = b_toronto_grouped['Neighbourhood']

for ind in np.arange(b_toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(b_toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head(40)

### Cluster Neighbourhoods
I've tried to use different number of clusters and decided to keep the version with 5 clusters here:

In [ ]:
# set number of clusters
kclusters = 5

b_toronto_grouped_clustering = b_toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(b_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

To create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood:

In [ ]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

b_toronto_merged = b_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
b_toronto_merged = b_toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

b_toronto_merged.head()

To visualize the result:

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(b_toronto_merged['Latitude'], b_toronto_merged['Longitude'], b_toronto_merged['Neighbourhood'], b_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Next I'll examine each cluster and try to make some conclusions at the end of this notebook.

In [ ]:
b_toronto_merged.loc[b_toronto_merged['Cluster Labels'] == 0, b_toronto_merged.columns[[1] + list(range(5, b_toronto_merged.shape[1]))]]

In [ ]:
b_toronto_merged.loc[b_toronto_merged['Cluster Labels'] == 1, b_toronto_merged.columns[[1] + list(range(5, b_toronto_merged.shape[1]))]]

In [ ]:
b_toronto_merged.loc[b_toronto_merged['Cluster Labels'] == 2, b_toronto_merged.columns[[1] + list(range(5, b_toronto_merged.shape[1]))]]

In [ ]:
b_toronto_merged.loc[b_toronto_merged['Cluster Labels'] == 3, b_toronto_merged.columns[[1] + list(range(5, b_toronto_merged.shape[1]))]]

In [ ]:
b_toronto_merged.loc[b_toronto_merged['Cluster Labels'] == 4, b_toronto_merged.columns[[1] + list(range(5, b_toronto_merged.shape[1]))]]

### Some conclusions

The purpose of the exercise was to explore Toronto and some of its neighbourhoods, to try out clustering and to visualize the results.

The clustering itself would need to be improved in order to be able to describe the clusters in a meaningful way since 4 out of 5 clusters contain just one or two neighbourhoods while one of the clusters contains almost all the neighbourhoods. Since this kind of improvements are outside the scope of the task, I'll stop here.

Thank you for reviewing this notebook!